Felipe Veloso hito 4

In [1]:
import pandas as pd
from functions import printUniques , report_scores , fetch_features

import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (50,30)

In [2]:
df = pd.read_csv('students.csv','|').drop('Unnamed: 0', axis =1)

In [3]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,nulidade,U,GT3,A,4,4,at_home,teacher,...,4,3,"""4""",1,1,"""3""",6,5,6,6
1,GP,F,"""17""",U,GT3,T,1,1,at_home,other,...,5,3,"""3""",1,1,"""3""",4,5,5,6
2,GP,F,"""15""",U,LE3,T,1,1,at_home,other,...,4,3,"""2""",2,3,"""3""",10,zero,8,10
3,GP,F,"""15""",U,GT3,T,4,2,health,services,...,3,2,"""2""",1,1,"""5""",2,15,14,15
4,GP,F,sem validade,U,GT3,T,3,3,other,other,...,4,3,"""2""",1,2,"""5""",4,6,10,10


In [4]:
#df_filter = df.replace(['zero','nulidade','sem validade','"17"','"15"','"16"','"19"','"18"','"22"','"20"','"21"','"3"', '"5"' ,'"1"' ,'"2"','2' ,'"4"','yes','no','1','3','4','5','6','7','8','9','10','F','M','0','16','14','25','12','54','18','26','20','56','24','28','13','15','22','21','11','17','19','75','30','38','40','23'],[np.nan,np.nan,np.nan,17,15,16,19,18,22,20,21,3,5,1,2,2,4,1,0,1,3,4,5,6,7,8,9,10,1,0,0,16,14,25,12,54,18,26,20,56,24,28,13,15,22,21,11,17,19,75,30,38,40,23])
df_filter = df.replace(['zero','nulidade','sem validade'],np.nan)
df_filter['age'] = df_filter['age'].str.replace(r'["]', '').astype('float')
df_filter['Medu'] = df_filter['Medu'].str.replace(r'["]', '').astype('float')
df_filter['Fedu'] = df_filter['Fedu'].str.replace(r'["]', '').astype('float')
df_filter['traveltime'] = df_filter['traveltime'].str.replace(r'["]', '').astype('float')
df_filter['studytime'] = df_filter['studytime'].str.replace(r'["]', '').astype('float')
df_filter['failures'] = df_filter['failures'].str.replace(r'["]', '').astype('float')
df_filter['famrel'] = df_filter['famrel'].str.replace(r'["]', '').astype('float')
df_filter['freetime'] = df_filter['freetime'].str.replace(r'["]', '').astype('float')
df_filter['goout'] = df_filter['goout'].str.replace(r'["]', '').astype('float')
df_filter['Dalc'] = df_filter['Dalc'].str.replace(r'["]', '').astype('float')
df_filter['Walc'] = df_filter['Walc'].str.replace(r'["]', '').astype('float')
df_filter['health'] = df_filter['health'].str.replace(r'["]', '').astype('float')
df_filter['absences'] = df_filter['absences'].str.replace(r'["]', '').astype('float')
df_filter['G1'] = df_filter['G1'].str.replace(r'["]', '').astype('float')
df_filter = df_filter.replace(['yes','no','F','M'],[1,0,0,1])

In [5]:
df_filter = df_filter.replace(['GP','U','GT3','A'],1)
df_filter = df_filter.replace(['MS','R','LE3','T'],0)

In [6]:
df_filter['Mjob'] = df_filter['Mjob'].replace(['at_home' ,'health', 'other' ,'services','teacher'],[1,2,3,4,5])

In [7]:
df_filter['Fjob'] = df_filter['Fjob'].replace(['teacher' ,'other' ,'services' ,'health' ,'at_home'],[1,2,3,4,5])
df_filter['reason'] = df_filter['reason'].replace(['course' ,'other' ,'home' ,'reputation'],[1,2,3,4])
df_filter['guardian'] = df_filter['guardian'].replace(['mother', 'father' ,'other'],[1,2,3])

In [8]:
df_filter.G2 = df_filter.G2.astype(float)
df_filter.G3 = df_filter.G3.astype(float)
df_filter.Fjob = df_filter.Fjob.astype(float)
df_filter = df_filter.replace(np.nan,-1)

In [9]:
df_filter = df_filter.astype(int)

In [10]:
df_filter = df_filter.replace(-1,np.nan)

In [11]:
df_filter.dropna(how='any', inplace=True)

# La preparación del ambiente de trabajo (imports, separación de muestras) para implementar modelos de predicción

In [12]:
#modelo_g1fix = smf.ols('G1 ~ failures+schoolsup+famsup+goout+studytime',df_filter).fit()
#modelo_g2fix = smf.ols('G2 ~ failures+goout+health+schoolsup+famsup+address',df_filter).fit()
#modelo_g3fix = smf.ols('G3 ~ failures+goout+Walc+freetime',df_filter).fit()

In [13]:
y_g1 = df_filter['G3']
params_g1 = ['failures','schoolsup','famsup','goout','studytime']
X_g1 = df_filter.loc[:,params_g1]
X_train_g1, X_test_g1 , y_train_g1 , y_test_g1 = train_test_split(X_g1 ,y_g1,test_size=.20 , random_state=3322)

In [14]:
g1_model = LinearRegression(fit_intercept=True,normalize=True).fit(X_train_g1,y_train_g1).predict(X_test_g1)

In [15]:
#Predecire con los regresores seleccionados para G1, asi ver la verosimilitud de los intervalos de confianza

In [16]:
y_g2 = df_filter['G3']
params_g2 = ['failures','goout','health','schoolsup','famsup','address']
X_g2 = df_filter.loc[:,params_g2]
X_train_g2, X_test_g2 , y_train_g2 , y_test_g2 = train_test_split(X_g2 ,y_g2,test_size=.20 , random_state=3322)

In [17]:
g2_model = LinearRegression(fit_intercept=True,normalize=True).fit(X_train_g2,y_train_g2).predict(X_test_g2)

In [18]:
#Predecire con los regresores seleccionados para G2, asi ver la verosimilitud de los intervalos de confianza

In [19]:
y_g3 = df_filter['G3']
params_g3 = ['failures','goout','Walc','freetime']
X_g3 = df_filter.loc[:,params_g3]
X_train_g3, X_test_g3 , y_train_g3 , y_test_g3 = train_test_split(X_g3,y_g3,test_size=.20 , random_state=3322)

In [20]:
g3_model = LinearRegression(fit_intercept=True,normalize=True).fit(X_train_g3,y_train_g3).predict(X_test_g3)

Obteniendo metricas de G3

In [21]:
report_scores(y_test_g1,g1_model)

17.6 -0.0


In [22]:
report_scores(y_test_g2,g2_model)

17.3 -0.0


In [23]:
report_scores(y_test_g3,g3_model)

18.2 -0.1


el modelo G3 con sus propios regresores tiene un mejor Error Cuadratico

In [25]:
fetch_features(df_filter,'G3')

,corr,abs_corr
attribute,,
G2,0.902984,0.902984
G1,0.803384,0.803384
failures,-0.345628,0.345628
goout,-0.221752,0.221752
Medu,0.180326,0.180326
higher,0.162044,0.162044
age,-0.132134,0.132134
reason,0.126877,0.126877
address,0.119397,0.119397


In [32]:
y_g4 = df_filter['G3']
params_g4 = ['G1','G2']
X_g4 = df_filter.loc[:,params_g4]
X_train_g4, X_test_g4 , y_train_g4 , y_test_g4 = train_test_split(X_g4,y_g4,test_size=.20 , random_state=3322)

In [33]:
g4_model = LinearRegression(fit_intercept=True,normalize=True).fit(X_train_g4,y_train_g4).predict(X_test_g4)

In [34]:
report_scores(y_test_g4,g4_model)

4.0 0.8


No permite inferir mucho los resultados del 1 semestre y el 2 semestre para saber las notas finales

# El reporte del mejor modelo predictivo en base a los resultados. 

El haber analizado los p valores a una confianza del `95%` nos entrego uno de los mejores FIT en la inferencia de notas finales del distintos alumnos, y en caso contrario solo predecir las notas del G1 y G2 para tener una respuesta del G3 fue totalmente innecesario, se evaluo por el metodo de error medio cruadratico, de todas formas, como estudio predictivo deja un tanto de insatisfaccion los resultados obtenidos, es posible que la informacion restante nos privara de puntos de vista valiosos.